<a href="https://colab.research.google.com/github/patricknelwan/ye-trump-GPT/blob/main/ye_trump_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers datasets torch accelerate peft bitsandbytes sentencepiece
!pip install torch-pruning
!pip install git+https://github.com/huggingface/peft
!pip install einops

!apt-get update && apt-get install build-essential

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 13.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe

In [2]:
files = ['kanye-quotes.txt', 'trump-quotes.txt']

with open('merged.txt', 'w', encoding='utf-8') as outfile:
    for fname in files:
        try:
            with open(fname, 'r', encoding='utf-8') as infile:
                content = infile.read()
                outfile.write(content + "\n\n")
                print(f"✅ Added {fname} to merged dataset")
        except FileNotFoundError:
            print(f"[!] {fname} not found - skipping")

    print("🎤🇺🇸 Kanye + Trump dataset merged successfully!")

with open('merged.txt', 'r', encoding='utf-8') as f:
    content = f.read()
    print(f"📊 Total characters: {len(content):,}")
    print(f"📄 Total lines: {len(content.splitlines()):,}")
    print(f"\n📝 Preview (first 200 characters):")
    print(content[:200] + "...")


✅ Added kanye-quotes.txt to merged dataset
✅ Added trump-quotes.txt to merged dataset
🎤🇺🇸 Kanye + Trump dataset merged successfully!
📊 Total characters: 14,182
📄 Total lines: 424

📝 Preview (first 200 characters):
"People always say that you can't please everybody. I think that's a cop-out. Why not attempt it? Cause think of all the people that you will please if you try."

"If I got any cooler I would freeze t...


In [3]:
from transformers import GPT2Tokenizer , GPT2LMHeadModel , Trainer , TrainingArguments
from datasets import load_dataset

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token =tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("gpt2")

dataset = load_dataset('text' , data_files = 'merged.txt')
def tokenize_function(examples) :
  return tokenizer(examples["text"] , truncation = True , padding = True , max_length = 128)

tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=2)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map (num_proc=2):   0%|          | 0/424 [00:00<?, ? examples/s]

In [4]:
!pip install -U bitsandbytes

In [6]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir="gpt2-ye-trump",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    save_steps=100,
    logging_steps=50,
    learning_rate=3e-4,
    weight_decay=0.01,
    fp16=True,
    logging_dir="logs",
    report_to="none",
    save_total_limit=1,

    disable_tqdm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
)

print("training..")
trainer.train()

model.save_pretrained("gpt2-ye-trump")
tokenizer.save_pretrained("gpt2-ye-trump")

print("saved to 'gpt2-ye-trump'")

training..


Step,Training Loss
50,1.429000


saved to 'gpt2-ye-trump'


In [7]:
import torch
from transformers import GPT2LMHeadModel, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = GPT2LMHeadModel.from_pretrained(
    "gpt2-ye-trump",
    device_map="auto",
    quantization_config=bnb_config,
    dtype=torch.float16,
)


In [8]:
import torch
import torch.nn as nn

model = GPT2LMHeadModel.from_pretrained("gpt2-ye-trump")
model.eval()
weighsToSparce = 0.5
for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        with torch.no_grad():
            w = module.weight.data
            mask = torch.rand(w.shape) > weighsToSparce
            w *= mask.to(w.device)

print("Model weights sparsified (" , weighsToSparce , ") zeroed out)")

model.save_pretrained("gpt2-ye-trump-sparse")

Model weights sparsified ( 0.5 ) zeroed out)


In [9]:
!pip install mistral-common


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 38.7 MB/s eta 0:00:00


In [10]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2-ye-trump")
tokenizer.save_pretrained("gpt2-ye-trump")

!rm -rf llama.cpp
!git clone https://github.com/ggerganov/llama.cpp
!pip install sentencepiece tokenizers gguf mistral-common

!cd llama.cpp && mkdir -p build && cd build && \
cmake .. \
-DCMAKE_BUILD_TYPE=Release \
&& make -j

!cp -r gpt2-ye-trump llama.cpp/models/gpt2-ye-trump

!cd llama.cpp && python3 convert_hf_to_gguf.py models/gpt2-ye-trump --outfile models/gpt2-ye-trump.gguf

!cd llama.cpp && ./build/bin/llama-quantize models/gpt2-ye-trump.gguf models/gpt2-ye-trump-Q2_K.gguf Q2_K

!ls -lh llama.cpp/models/*.gguf



Cloning into 'llama.cpp'...
remote: Enumerating objects: 63036, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 63036 (delta 98), reused 42 (delta 39), pack-reused 62877 (from 4)
Receiving objects: 100% (63036/63036), 157.41 MiB | 13.78 MiB/s, done.
Resolving deltas: 100% (45727/45727), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 6.1 MB/s eta 0:00:00
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git 

In [29]:
!cd llama.cpp && ./build/bin/llama-cli -m models/gpt2-ye-trump-Q2_K.gguf -p "hole " -n 100 --temp 1 --no-warmup --repeat_penalty 2.0

build: 6585 (b05a9d65) with cc (Ubuntu 11.4.0-1ubuntu1~22.04.2) 11.4.0 for x86_64-linux-gnu
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 22 key-value pairs and 148 tensors from models/gpt2-ye-trump-Q2_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gpt2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Gpt2 Ye Trump
llama_model_loader: - kv   3:                         general.size_label str              = 124M
llama_model_loader: - kv   4:                           gpt2.block_count u32              = 12
llama_model_loader: - kv   5:                        gpt2.context_length u32              = 1024
lla